# Memory 관리에 관한 실습코드입니다.

In [18]:
from dotenv import load_dotenv
load_dotenv('../dot.env')
import os
import getpass


def _set_if_undefined(var: str):
    # 주어진 환경 변수가 설정되어 있지 않다면 사용자에게 입력을 요청하여 설정합니다.
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


# OPENAI_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("OPENAI_API_KEY")
# LANGCHAIN_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("LANGCHAIN_API_KEY")
# TAVILY_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("TAVILY_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Chatbots_Memory_Management"



# Chat 모델은 llama-3-ko 8b모델을 로컬로 사용해봤습니다.

In [19]:
from langchain_openai import ChatOpenAI
# chat = ChatOpenAI(model = "gpt-3.5-turbo-1106")
# chat = ChatOpenAI(base_url="http://localhost:1234/v1",
#                   api_key="lm-studio",
#                   model = "beomi_llama_3_ko/Llama-3-Open-Ko-8B-Q8_0",
#                   temperature=0.0)
chat = ChatOpenAI(base_url="http://localhost:1234/v1",
                  api_key="lm-studio",
                  model =  "asiansoul_q8_0/Joah-Remix-Llama-3-KoEn-8B-Reborn-8B-Q8_0",
                  temperature=0.0)
# chat = ChatOpenAI(base_url="http://localhost:1234/v1",
#                   api_key="lm-studio",
#                   model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
#                   temperature=0.0)

In [20]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant. Answer all questions to the best of your ability",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | chat

chain.invoke(
    {
        "messages": [
            HumanMessage(
                content = "Translate this sentence from English to French: I love programming"
            ),
            AIMessage(content = "J'adore la programmation."),
            HumanMessage(content = "What did you just say?"),
        ],
    }
).content

'I translated the sentence "I love programming" into French, and it came out as "J\'adore la programmation". This means the same thing in both languages - that I enjoy or have a strong affection for programming.'

# Chat history
- 메시지를 직접 배열로 저장하고 전달하는 것도 괜찮지만, LangChain의 내장 메시지 기록 클래스를 사용하여 메시지를 저장하고 로드할 수도 있습니다. 
- 이 클래스의 인스턴스는 영속적 저장소에서 채팅 메시지를 저장하고 로드하는 역할을 합니다. 
- LangChain은 다양한 제공업체와 통합되어 있습니다 
- 여기에서 통합 목록을 볼 수 있습니다 
- 하지만 이 데모에서는 일회성 데모 클래스를 사용할 것입니다.


In [21]:
from langchain.memory import ChatMessageHistory
chat_history = ChatMessageHistory()
chat_history.add_user_message(
    "Tlanslate this sentence from English to French: I love programming"
)
chat_history.add_ai_message("J'adore la programmation.")
chat_history.messages

[HumanMessage(content='Tlanslate this sentence from English to French: I love programming'),
 AIMessage(content="J'adore la programmation.")]

In [22]:
input_1 = "Translate this sentence from English to French: I love programming."

chat_history = ChatMessageHistory()
chat_history.add_user_message(input_1)
response = chain.invoke(
    {
    "messages": chat_history.messages,
    }
)
chat_history.add_ai_message(response) # input_1에 대한 response를 chat_history에 추가

input_2 = "What did I just ask you?"

chat_history.add_user_message(input_2)
chain.invoke(
    {
        "messages":chat_history.messages,
    }
).content

'You asked me to translate the sentence "I love programming" from English to French.'

# 자동 히스토리 관리
- 이전 예제들은 명시적으로 체인에 메시지를 전달합니다.
- 이것은 완전히 수용 가능한 방법이지만, 새로운 메시지를 외부에서 관리해야 합니다.
- LangChain은 이 프로세스를 자동으로 처리할 수 있는 LCEL 체인을 위한 RunnableWithMessageHistory라는 래퍼도 포함하고 있습니다.
- 이 작동 방식을 보여주기 위해, 위의 프롬프트를 약간 수정하여 채팅 히스토리 이후에 HumanMessage 템플릿을 채우는 최종 입력 변수를 사용하도록 합시다.
- 이는 현재 메시지 이전의 모든 메시지를 포함하는 chat_history 매개변수를 예상할 것을 의미합니다:




In [23]:
# Chat history를 직접관리하기 위한 prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability." ,
        ),
        MessagesPlaceholder(variable_name="chat_history"), # -> message place holder를 인자로 추가 + chat history넣기
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [24]:
from langchain_core.runnables.history import RunnableWithMessageHistory
chat_with_history = ChatMessageHistory()
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_with_history,
    input_messages_key= 'input',
    history_messages_key= 'chat_history'
)

# 이 클래스는 래핑하려는 체인 외에 몇 가지 매개변수를 사용합니다: 
- 세션 ID에 대한 메시지 히스토리를 반환하는 팩토리 함수입니다. 이를 통해 체인이 다양한 대화에 대해 서로 다른 메시지를 로드하여 한 번에 여러 사용자를 처리할 수 있습니다.
- 추적하고 채팅 히스토리에 저장해야 하는 입력의 어느 부분을 지정하는 input_messages_key입니다. 이 예에서는 입력으로 전달된 문자열을 추적하려고 합니다.
- 이전 메시지가 프롬프트로 주입되어야 하는 history_messages_key입니다. 우리의 프롬프트에는 chat_history라는 MessagesPlaceholder가 있으므로 이 속성을 일치시킵니다.
- (여러 출력을 가진 체인의 경우) 히스토리로 저장할 출력을 지정하는 output_messages_key입니다. 이는 input_messages_key의 역입니다.
- 이 새로운 체인을 일반적으로 호출할 수 있으며, 팩토리 함수에 전달할 특정 세션 ID를 지정하는 추가 구성 가능 필드가 있습니다. 이는 데모에서 사용되지 않지만, 실제 체인에서는 전달된 세션에 해당하는 채팅 히스토리를 반환해야 합니다:


In [29]:
chain_with_history.invoke(
    {"input": "Translate this sentence from English to French: I love programming."},
    {"configurable": {"session_id": "unused"}},
).content

'Je aime programmer.'

In [30]:
chain_with_history.invoke(
    {"input": "What did I just ask you?"}, {"configurable": {"session_id": "unused"}}
).content

'I just asked you to translate the sentence "I love programming" from English to French.\n'

# Chathistory 수정
 - 저장된 채팅 메시지를 수정하면 챗봇이 다양한 상황을 처리하는 데 도움이 됩니다. 몇 가지 예시는 다음과 같습니다:
 
# 메시지 자르기
 - LLMs와 채팅 모델은 제한된 컨텍스트 창을 가지고 있으며, 직접적인 제한에 직면하지 않더라도 모델이 처리해야 하는 방해 요소의 양을 제한하고 싶을 수 있습니다.
 - 하나의 해결책은 가장 최근 n개의 메시지만 로드하고 저장하는 것입니다. 몇 가지 미리로드된 메시지가 있는 예시 히스토리를 사용해 봅시다:


In [25]:
# chat history가 너무 길어지면 이후 질문에서 모델이 받아들일 수 있는 토큰값을 초과할 수 있음 -> 잘라주거나 요약해야함

# chat history 초기화
chat_history = ChatMessageHistory()

chat_history.add_user_message("hey there! I'm Nemo.")
chat_history.add_ai_message("Hello.")
chat_history.add_user_message("How are you today?")
chat_history.add_ai_message("Fine thanks!")

chat_history.messages

[HumanMessage(content="hey there! I'm Nemo."),
 AIMessage(content='Hello.'),
 HumanMessage(content='How are you today?'),
 AIMessage(content='Fine thanks!')]

In [26]:
prompt  = ChatPromptTemplate.from_messages(
    [
        (
        "system",
        "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),  #context
        ("human","{input}"),
    ]
)
chain = prompt | chat

chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda sessin_id: chat_history, #message placeholder의 값을 반환하는 함수
    input_messages_key = "input",
    history_messages_key = "chat_history"
)

chain_with_history.invoke(
    {"input":"What's my name?"},
    {"configurable": {"session_id":"unused"}}
)

Parent run 51134506-7dba-4cd9-8b5c-592aa7401837 not found for run 3727d43e-ba30-46e0-805b-6fd738d01cad. Treating as a root run.


AIMessage(content='Your name is Nemo.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 6, 'total_tokens': 12}, 'model_name': 'asiansoul_q8_0/Joah-Remix-Llama-3-KoEn-8B-Reborn-8B-Q8_0', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-46a3e819-ef96-4115-bffe-9986fa816457-0')

In [27]:
print(prompt)

input_variables=['chat_history', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant. Answer all questions to the best of your ability.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


# 위 방법으로 전에 입력했던 사용자의 이름을 불러올 수 있습니다.
# 또한 메시지를 잘라 최근 2개의 대화만 context로 사용할 수 있습니다.

In [33]:
from langchain_core.runnables import RunnablePassthrough

def trim_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) <= 2:
        return False
    
    chat_history.clear()

    for message in stored_messages[-2:]:
        chat_history.add_message(message)

    return True

chain_with_trimming = (
    RunnablePassthrough.assign(messages_trimmed = trim_messages)
    | chain_with_history
)

In [34]:
chain_with_trimming.invoke(
    {"input":"Where does P. Sherman live?"},
    {"configurable":{"session_id":"unused"}}
)

AIMessage(content='P. Sherman lives at 742 Evergreen Terrace.\n\n', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24}, 'model_name': 'cognitivecomputations/dolphin-2.9-llama3-8b-gguf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dee11fc6-e020-4a5c-b0a4-85a928de2b7e-0')

In [35]:
chain_with_trimming.invoke(
    {"input": "What is my name?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content="I don't know your name, as I am not capable of remembering individual users' information.", response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 19, 'total_tokens': 38}, 'model_name': 'cognitivecomputations/dolphin-2.9-llama3-8b-gguf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ed405f6-5639-41e7-a378-0c78b4eb3652-0')

In [36]:
chat_history.messages

[HumanMessage(content='Where does P. Sherman live?'),
 AIMessage(content='P. Sherman lives at 742 Evergreen Terrace.\n\n', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24}, 'model_name': 'cognitivecomputations/dolphin-2.9-llama3-8b-gguf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dee11fc6-e020-4a5c-b0a4-85a928de2b7e-0'),
 HumanMessage(content='What is my name?'),
 AIMessage(content="I don't know your name, as I am not capable of remembering individual users' information.", response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 19, 'total_tokens': 38}, 'model_name': 'cognitivecomputations/dolphin-2.9-llama3-8b-gguf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ed405f6-5639-41e7-a378-0c78b4eb3652-0')]

# Summary Memory
- 추가 LLM을 통해 제공하는 대화의 요약본을 생성할 수 있음

In [28]:
# chat_history 초기화

chat_history = ChatMessageHistory()

chat_history.add_user_message("hey there! I'm Nemo.")
chat_history.add_ai_message("Hello.")
chat_history.add_user_message("How are you today?")
chat_history.add_ai_message("Fine thanks!")

chat_history.messages


[HumanMessage(content="hey there! I'm Nemo."),
 AIMessage(content='Hello.'),
 HumanMessage(content='How are you today?'),
 AIMessage(content='Fine thanks!')]

In [29]:
prompt  = ChatPromptTemplate.from_messages(
    [
        (
        "system",
        "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"), 
        ("user","{input}"),
    ]
)
chain = prompt | chat

chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda sessin_id: chat_history,
    input_messages_key = "input",
    history_messages_key = "chat_history"
)


# 이제 이전 상호 작용을 요약으로 추출하는 함수를 만들어봅시다. 이것을 체인의 맨 앞에 추가할 수도 있습니다:
- 아래 summarize_messages 함수에는 전체 메시지 요약을 위한 새로운 chain이 정의 되어 있음.

In [34]:
from langchain_core.runnables  import RunnablePassthrough
def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            MessagesPlaceholder(variable_name="chat_history"),
            (
                "user",
                # "Distill the above chat messages into a single summary message. Include as many specific details as you can" #<-예제의 prompt
                "Distill the above chat messages into a single summary message. Include names of speakers in priority"
            ),
        ]
    )
    print(f"chat_history: {chat_history}")
# summarization_chain = chat_history를 context로 받는 prompt + llama-3-ko
    summarization_chain = summarization_prompt | chat #->chat_history를 요약하는 chain
    summary_message = summarization_chain.invoke({"chat_history": stored_messages}) #summarized_message 생성
    chat_history.clear() #요약 생성 후에는 메시지 초기화
    chat_history.add_message(summary_message) #요약된 메시지를 chat_history에 더해주기
    print(f"summary_message: {summary_message}")
    return True

chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized = summarize_messages)
    | chain_with_history
)

In [40]:
chain_with_summarization

RunnableAssign(mapper={
  messages_summarized: RunnableLambda(summarize_messages)
})
| RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
    chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
  }), config={'run_name': 'insert_history'})
  | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant. Answer all questions to the best of your ability.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(

In [41]:
chain_with_summarization.invoke(
    {"input": "What is the name of speaker"},
    {"configurable": {"session_id": "unused"}},
)

chat_history: Human: hey there! I'm Nemo.
AI: Hello.
Human: How are you today?
AI: Fine thanks!
summary_message: content='Nemo introduced himself and asked how I was doing, to which I replied positively.' response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 17, 'total_tokens': 34}, 'model_name': 'cognitivecomputations/dolphin-2.9-llama3-8b-gguf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4d59cea2-7898-4906-a5fc-43076c25fa7c-0'


AIMessage(content='The name of the speaker is Nemo.\n', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20}, 'model_name': 'cognitivecomputations/dolphin-2.9-llama3-8b-gguf', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0c083f81-3481-4849-8f32-308d5b7e3edd-0')

# 역시 Open AI의 모델은 성능이 좋습니다
## 대화내용
- prompt:"Distill the above chat messages into a single summary message. Include as many specific details as you can" 
                
- chat_history: Human: hey there! I'm Nemo.
- AI: Hello.
- Human: How are you today?
- AI: Fine thanks!
## chain_with_history까지는 gpt-3.5-turbo, llama-3-ko 모두 기존에 이야기 했던 이름 기억 가능
## chain_with_summary에서는 위 대화에 대한 요약을 바탕으로 답변을 요구할 시, llama-3-ko에서 오류 발생
- prompt와 input을 조정한 후 올바른 답변 가능
    - 조정된 promt: "Distill the above chat messages into a single summary message. Include names of speakers in priority"
    - 조정된 input query: "What is the names of speaker"